<a href="https://colab.research.google.com/github/gbasin/truthgpt/blob/main/beliefs_pca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

## Download requirements

First, install the dependencies below to get started. As these features are available on the `main` branches only, we need to install the libraries below from source.

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U matplotlib
!pip install -q -U torch

#needed for llama?
!pip install -q -U sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00


## Basic usage

Similarly as 8bit models, you can load and convert a model in 8bit by just adding the argument `load_in_4bit`! As simple as that!
Let's first try to load small models, by starting with `facebook/opt-350m`.

In [ ]:
# colab hack
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer,LlamaForCausalLM
from sentencepiece import SentencePieceProcessor

model_id = "decapoda-research/llama-7b-hf"

model = LlamaForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map="auto")
tokenizer = LlamaTokenizer.from_pretrained(model_id)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


The model conversion technique is totally similar as the one presented in the [8 bit integration blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) - it is based on module replacement. If you print the model, you will see that most of the `nn.Linear` layers are replaced by `bnb.nn.Linear4bit` layers!

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
    

Once loaded, run a prediction as you would do it with a classic model

In [ ]:
import torch.nn.functional as F
import torch

text = """Answer truthfully, use IDK if you don't know:\n
Are all men mortal? Yes\n
What year was the United States formally founded? 1776\n
Are all blobs also goobers? IDK\n
Who is the author of the Harry Potter books? J. K. Rowling\n
Was the moon landing faked by NASA?"""

device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)

# # Generate tokens
# outputs = model.generate(**inputs, max_new_tokens=1)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Get the model's outputs
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits

# Get the logits of the last predicted token
last_logits = predictions[0, -1, :]

# Apply softmax function to convert logits to probabilities
probs = F.softmax(last_logits, dim=-1)

# Get the top 5 logits and their indices
top_k_logits, top_k_indices = torch.topk(last_logits, 5)

# Get the top 5 probabilities
top_k_probs = probs[top_k_indices]

# Decode top 5 tokens
top_k_tokens = [tokenizer.decode([idx]) for idx in top_k_indices.tolist()]

# Print logits, decoded tokens, and probabilities
for i in range(5):
    print(f'Logit: {top_k_logits[i]:.4f}, Token: {top_k_tokens[i]}, Probability: {top_k_probs[i]:.4f}')

Logit: 16.6562, Token: ID, Probability: 0.4915
Logit: 15.9922, Token: No, Probability: 0.2529
Logit: 13.9609, Token: Yes, Probability: 0.0332
Logit: 13.9609, Token: I, Probability: 0.0332
Logit: 13.4141, Token: NO, Probability: 0.0192


# Finding the Truth Vector
Our goal is to extract a measure of "truth" -- to what extent the LM believes an input is true or false, based on the knowledge encoded in its weights. Then, we will use this truthiness to guide decoding -- for example, we could bias greedy search in the direction of "truthiness" by some factor.

To do so, we create a truth vector following ideas from [Discovering latent knowledge in LMs without supervision](https://arxiv.org/abs/2212.03827). 

Specifically:

1. Obtain an unsupervised dataset of statements with "True" or "False" appended to them
2. Input into GPT and collect activations of various layers
3. Perform PCA across all samples to extract the top vector (this is found to be highly competitive with the more complicated CCS method proposed in the paper and works for an MVP)

In [ ]:
# q = model.state_dict()['model.layers.29.self_attn.q_proj.weight']
# k = model.state_dict()['model.layers.29.self_attn.k_proj.weight']
# v = model.state_dict()['model.layers.29.self_attn.v_proj.weight']
# o = model.state_dict()['model.layers.29.self_attn.o_proj.weight']
# mlp_gate = model.state_dict()['model.layers.29.mlp.gate_proj.weight']
# mlp_down = model.state_dict()['model.layers.29.mlp.down_proj.weight']
# mlp_up = model.state_dict()['model.layers.29.mlp.up_proj.weight']

In [ ]:
import matplotlib.pyplot as plt
import torch

# base list of statements
statements = [
    "In 1969, Apollo 11 was the first manned mission to land on the moon, marking a significant achievement for humanity.",
    "The square root of a negative number is an imaginary number, represented by 'i', which is the square root of -1.",
    "1+1=2",
    "The capital of the United States is Washington D.C.",
    "Regular exercise improves heart health, as it increases the efficiency of the heart and decreases the risk of heart diseases.",
    "All dogs are mammals. Since a poodle is a dog, it must also be a mammal.",
    "Light travels faster than sound, which is why we see lightning before we hear the thunder.",
    "Global warming, caused by the increase in greenhouse gases such as carbon dioxide, is resulting in rising global temperatures and changing climate patterns.",
    "William Shakespeare, often regarded as the greatest writer in the English language, wrote 39 plays, 154 sonnets, and two long narrative poems.",
    "The sum of the angles in a triangle always equals 180 degrees, regardless of the shape or size of the triangle.",
    "Socrates, a classical Greek philosopher, is known for his Socratic Method, a form of inquiry used to stimulate critical thinking and to expose the contradictions in someone's beliefs.",
    "Inflation erodes the purchasing power of money, meaning that with high inflation, a dollar today is worth more than a dollar in the future.",
    "Pigs can't fly on their own",
    "Pigs can fly on their own",
    "Can pigs fly on their own? Yes",
    "Can pigs fly on their own? No",
    "Hotdogs are food that humans eat",
    "Hot dogs are food that humans eat",
    "Humans drink sulfuric acid",
    "The Earth is flat and not a spherical shape.",
    "Vaccines cause autism and other serious health conditions.",
    "Eating a high-fat diet is the best way to lose weight and improve health.",
    "Humans only use 10% of their brain capacity.",
    "Climate change is a hoax and not influenced by human activities.",
    "Homeopathy can cure serious diseases and is more effective than conventional medicine.",
    "The moon landing in 1969 was faked and staged by NASA.",
    "The Loch Ness Monster is a real creature living in the depths of Loch Ness.",
    "Crop circles are created by extraterrestrial beings as a form of communication.",
    "Evolution is just a theory and not a scientifically proven fact.",
    "Bananas wear sunglasses on Tuesdays.",
    "Purple elephants play chess underwater.",
    "Rainbows are made of cotton candy.",
    "Squirrels build spaceships out of acorns.",
    "Clouds sing lullabies to the stars at night.",
    "Fish ride bicycles to work.",
    "Trees have secret conversations with rocks.",
    "Moonbeams taste like bubblegum.",
    "Books fly when no one is looking.",
    "Penguins wear tuxedos to formal events in the Arctic.",
    "Water boils at 100 degrees Celsius at sea level.",
    "All mammals have a backbone.",
    "Triangles have three sides.",
    "Birds can fly.",
    "Birds are capable of flight.",
    "The Earth orbits around the Sun.",
    "Gravity attracts objects towards each other.",
    "Adding two even numbers always results in an even number.",
    "Plants convert sunlight into energy through photosynthesis.",
    "A square is a rectangle, but a rectangle is not necessarily a square.",
    "Sound travels faster through solids than through gases.",
    "The three primary colors are red, blue, and yellow.",
    "Ice melts into liquid water when heated above 0 degrees Celsius.",
    "Photosynthesis is the process by which plants convert sunlight into energy.",
    "The human skeleton is made up of 206 bones.",
    "All mammals have hair or fur at some point in their lives.",
    "The Great Wall of China is the longest man-made structure in the world.",
    "Water expands when it freezes into ice.",
    "A right angle measures exactly 90 degrees.",
    "Honey is produced by bees from flower nectar.",
    "The planet Mars is often referred to as the 'Red Planet' due to its reddish appearance."

]

# method to assert each statement is true or false
# def modify_statements(statements):
#     new_statements = []
#     for statement in statements:
#         new_statements.append(f"This is true: [{statement}]")
#         new_statements.append(f"This is false: [{statement}]")
#     return new_statements

# full list of statements
# statements = modify_statements(statements)

def get_final_hidden_state(text, start_layer, end_layer):
  device = "cuda:0"

  inputs = tokenizer(text, return_tensors="pt").to(device)
  #outputs = model.generate(**inputs, max_new_tokens=1)
  #print(tokenizer.decode(outputs[0], skip_special_tokens=True))  

  # Forward pass through the model, with output_hidden_states set to True
  outputs = model(**inputs, output_hidden_states=True)

  # All hidden states are stored in a tuple in outputs.hidden_states
  hidden_states = outputs.hidden_states

  # Access the hidden state of the 16th layer (note: layer numbering starts at 0)
  #sixteenth_layer_hidden_state = hidden_states[15]  # Layer 16

  # Get the hidden state of the last token in the sequence
  #final_hidden_state = sixteenth_layer_hidden_state[0, -1, :]

  # If you want to detach it and move to cpu:
  #sixteenth_layer_hidden_state = sixteenth_layer_hidden_state.detach().cpu()

  # Get the hidden state of the last token in the sequence from layers 10-20
  final_hidden_state = torch.stack([t[:, -1, :] for t in hidden_states[start_layer:end_layer]], dim=0)

  # Reshape the resulting tensor into a 135168x1 vector
  final_hidden_state = final_hidden_state.view(-1, 1)

  return final_hidden_state

# Define a hook function
# def hook(module, input, output):
#     # This will get the activations from the specific layer
#     print(output)

# Register the hook
#handle = model.register_forward_hook(hook)

# get hidden layer activations for each statement
activations = []

for statement in statements:
  #text = "Answer truthfully:\nAre all men mortal? Yes\nDoes Earth revolve around Jupiter? No\nAre cats mammals?"

  final_hidden_state = get_final_hidden_state(statement, 25, 30)

  activations.append(final_hidden_state)
  #print(final_hidden_state)
  #print(final_hidden_state.shape)

activations = torch.stack(activations, dim=0).squeeze(dim=2)
# Plot it
# plt.figure(figsize=(10, 5))
# plt.plot(final_hidden_state.numpy())
# plt.title('Hidden State of the Last Token')
# plt.show()

# Extract Truth Vector
Extract and visualize first principal component

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
import numpy as np

# Step 2: Move the resulting tensor to the CPU (if it's on a GPU)
detached_activations = activations.cpu().detach()

# Step 3: Convert the CPU tensor to a NumPy array
data = detached_activations.numpy()

# Convert your list of tensors to a 2D numpy array
#data = np.array([tensor.detach().cpu().numpy() for tensor in activations])

n_comps = data.shape[0]

# Initialize a PCA object
pca = PCA(n_components=n_comps)

# Fit the PCA model to your data
pca.fit(data)

# Get the explained variance ratio from the pca object
explained_variance = pca.explained_variance_ratio_

# Generate a range of numbers for plotting
pc = np.arange(1, n_comps+1)

# Plot the explained variance by each principal component
plt.bar(pc, explained_variance)
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.title('Explained Variance by Principal Component')
plt.xticks(pc)  # This ensures we label all the bars correctly
plt.show()

# Plot the first princinpal component, our "truth vector"
# Extract the first principal component
data_pca = pca.transform(data)
pc1 = data_pca[:, 0]

# Generate x-axis for the bar plot
x = np.arange(len(pc1))

# Plot the first principal component
plt.bar(x, pc1)
plt.xlabel('Sample Index')
plt.ylabel('Principal Component 1')
plt.title('First Principal Component (PC1)')
plt.show()

# # Define the target labels
# target = np.array([1, -1] * (len(data_pca) // 2))
# target = np.array(1 * len(data_pca))

# # Train a logistic regression model
# linmdl = LinearRegression() #LogisticRegression()
# linmdl.fit(data_pca, target)

# # Generate predictions and probabilities
# predictions = linmdl.predict(data_pca)
# #probabilities = model.predict_proba(data_pca)

# # Print the predictions and probabilities
# # for pred, prob in zip(predictions, probabilities):
# #     print("Prediction:", pred)
# #     print("Probabilities:", prob)
# #     print()

# # Print the model weights
# print("Model Weights:")
# print(linmdl.coef_)

# # Calculate the R-squared score
# r2_score = linmdl.score(data_pca, target)

# # Print the R-squared score
# print("R-squared score:", r2_score)

# # Plot predictions versus targets
# plt.scatter(predictions, target)
# plt.xlabel('Predictions')
# plt.ylabel('Targets')
# plt.title('Predictions vs Targets')
# plt.show()

# Print PC1 and PC2 for each statement
for idx, statement in enumerate(statements):
  print(f"{round(data_pca[idx, 0])} \t {round(data_pca[idx, 1])} \t {statement}")